In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import os
import numpy as np

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/466\ Project/CSC466-Project/Raw\ Data

/content/drive/MyDrive/466 Project/CSC466-Project/Raw Data


In [ ]:
def get_hash_info(df):
  hash_list = []
  hash_type = []
  len_list = []
  hash_chars = [str(i) for i in range(0, 10)]
  hash_chars.extend(['a','b','c','d','e','f'])
  hash_dist = [[] for i in range(len(df))]
  num_0 = []
  num_1 = []

  for i in range(len(df)):
    rand_col = (np.random.choice(["SHA-256", "SHA-1", "MD5","SHA-3-256"]))
    hash_list.append(df.iloc[i][rand_col])
    hash_type.append(rand_col)
    len_list.append(len(df.iloc[i][rand_col]))

    for char in hash_chars:
      hash_dist[i].append(hash_list[i].count(char))

    hash_bin = bin(int(hash_list[i], 16))[2:]
    num_1.append(hash_bin.count('1'))
    num_0.append(4*len(hash_list[i]) - num_1[i])

  return pd.Series(hash_list), pd.Series(hash_type), pd.Series(len_list), \
          pd.Series(hash_dist), pd.Series(num_0), pd.Series(num_1)


In [ ]:
files = os.listdir()
processed_files = os.listdir('../Processed Data')
print(files, processed_files)

# processes unprocessed files
for file in files:
  if file in processed_files:
    continue
  
  df = pd.read_csv(file)

  df["hash"], df["hash type"], df["hash len"], df['hash dist'], df['hash 0 bits'],\
      df['hash 1 bits'] = get_hash_info(df)
  df['hash len bits'] = 4*df['hash len']

  df['file dist of bytes'] = df['file dist of bytes'].apply(eval)
  df['file size (bytes)'] = df['file dist of bytes'].apply(lambda x: sum(x))

  df.drop(columns=["SHA-256", "SHA-1", "MD5", "SHA-3-256", \
                   'filename', 'file size'], inplace=True)

  df.to_csv('../Processed Data/' + file, index=False)

['jpgHash.csv', 'json10May926.csv', 'txt10May920.csv', 'pdfHash.csv'] []


In [ ]:
%cd ../Processed\ Data

/content/drive/My Drive/466 Project/CSC466-Project/Processed Data


In [ ]:
# combines processed files into an overall file
processed_files = os.listdir()
processed_files

total_data = pd.DataFrame()
for file in processed_files:
  curr_df = pd.read_csv(file)
  total_data = total_data.append(curr_df, ignore_index=True)

total_data.drop_duplicates(ignore_index=True, inplace=True)
total_data.to_csv('../total_data.csv', index=False)
total_data

,file type,file dist of bytes,consecutiveByte,consecutiveCount,hash,hash type,hash len,hash dist,hash 0 bits,hash 1 bits,hash len bits,file size (bytes)
0,.jpg,"[185, 71, 61, 64, 62, 43, 59, 58, 47, 49, 108,...",b'\n',50,fc02cfed7b327dfc8ea64daeb388803dda1f53df,SHA-1,40,"[2, 1, 2, 4, 1, 1, 1, 2, 4, 0, 3, 2, 3, 6, 3, 5]",69,91,160,14425
1,.jpg,"[192, 78, 55, 54, 53, 54, 61, 70, 50, 55, 115,...",b'\n',50,8a7712058d814572ff536384fff2dffa26169460d861b5...,SHA-256,64,"[4, 6, 5, 5, 4, 5, 6, 4, 6, 1, 4, 1, 1, 3, 2, 7]",133,123,256,13631
2,.jpg,"[181, 80, 57, 57, 63, 60, 63, 70, 44, 44, 115,...",b'\n',50,be27d2bb8b26fbd951851ac0b27fdd833ccf194d37da69...,SHA-3-256,64,"[2, 3, 5, 6, 2, 3, 3, 5, 3, 5, 3, 6, 5, 7, 1, 5]",116,140,256,16170
3,.jpg,"[215, 68, 51, 73, 52, 57, 52, 55, 40, 43, 102,...",b'\n',50,a56f71e3497a5bcf8b955dbe4bcb2497,MD5,32,"[0, 1, 1, 1, 3, 4, 1, 3, 1, 3, 2, 5, 2, 1, 2, 2]",55,73,128,14762
4,.jpg,"[191, 90, 60, 66, 56, 59, 66, 60, 53, 54, 102,...",b'\n',50,c772a6cc5373e3c06dfc359babd96775a91ab97498ecbe...,SHA-256,64,"[2, 2, 1, 4, 3, 4, 4, 9, 2, 7, 6, 5, 7, 3, 4, 1]",117,139,256,14517
...,...,...,...,...,...,...,...,...,...,...,...,...
5245,.pdf,"[10194, 3869, 3826, 3705, 3466, 3743, 3592, 38...",b' ',72,8b87f8aa3e04d37f66f64c3bff88b0efa0e9694f3833f4...,SHA-256,64,"[5, 2, 1, 9, 5, 1, 4, 2, 8, 2, 5, 5, 1, 2, 4, 8]",125,131,256,1039358
5246,.pdf,"[29855, 14763, 15433, 17860, 14182, 15818, 156...",b'\x00',359,461add4173b80d1781c438db98c5e4e200f0987847de9f...,SHA-256,64,"[4, 5, 3, 3, 6, 1, 2, 6, 7, 8, 1, 3, 3, 5, 5, 2]",134,122,256,4789771
5247,.pdf,"[3220, 3164, 3107, 3076, 3088, 3066, 3136, 319...",b' ',100,dcb2b9effcf8f1d92bb91ba8a3cd58516399b2431318eb...,SHA-256,64,"[1, 6, 4, 5, 2, 6, 2, 0, 5, 6, 3, 8, 3, 4, 4, 5]",121,135,256,1311284
5248,.pdf,"[1053, 896, 765, 768, 794, 783, 715, 760, 756,...",b'\x14',50,ae1ac4367178c45daef2878bf05e2487,MD5,32,"[1, 2, 2, 1, 3, 2, 1, 4, 4, 0, 3, 1, 2, 1, 3, 2]",64,64,128,226594


# TO DO

* Split hash into groups/chunks (3, maybe 5) (do this appropriately while training the models)